In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from edmAnalyzer import *



import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.optimize import curve_fit

In [4]:
run = 10
sequence = 161
offset = 19
number_of_blocks = 4
quantity_of_interest = "Readout_LA_Clippers"
degeneracy = "D"

In [5]:

sequence = str(sequence).zfill(4)
run = str(run).zfill(4)

folderpath = r"C:\ACME_analysis\doppler"+run+"."+ sequence 
list_of_target_folders = [folderpath]
xx=hh.headerHandler(list_of_target_folders)



df = xx[[quantity_of_interest]]   # Change this

df.to_csv(folderpath+r"\\"+r"\TiSapph_N_P.csv")
print(len(df))

def touch(folder_path):
    """
    Rename files in the specified folder from xxxx.xxxx.xxxx.bin to xxxx.xxxx.xxxx.0000.bin
    """
    for filename in os.listdir(folder_path):
        if filename.endswith('.bin') and len(filename) == 18 and filename.count('.') == 3:
            # filename is of the form xxxx.xxxx.xxxx.bin
            new_filename = filename.replace('.bin', '.0000.bin')
            source_path = os.path.join(folder_path, filename)
            target_path = os.path.join(folder_path, new_filename)

            if not os.path.exists(target_path):
                os.rename(source_path, target_path)
            else:
                print(f"Collision detected: {target_path} already exists, skipping.")

touch(folderpath)
bdf = pd.DataFrame()
bdf['Polarization Switching XY Swapped'] = [0,0,0,0,0]
NN = []
N = []
AA = []
XX = []
YY = []



128


In [6]:
for i in range(number_of_blocks):
    a = binCalculator(folderpath+r"\\"+run+"."+sequence + "." + str(i).zfill(4)+r".0000.bin", r"C:\ACMEcode\ACMEanalysis\templates\acmeiii\offsettrace\offsettrace"+str(offset)+r".json", bdf)
    a.default_pipeline()
    NN.append(a.grouped_N_total)
    N.append(a.grouped_N_total)
    AA.append(a.binresult.A)
    XX.append(a.ungrouped_Fx[:,:,:,68*20:100*20].sum(axis = (1,2,3)))
    YY.append(a.ungrouped_Fy[:,:,:,68*20:100*20].sum(axis = (1,2,3)))
AA = np.concatenate(AA)
XX = np.concatenate(XX)
YY = np.concatenate(YY)
NN = [NN[i].sum(axis = (-1, -2)) for i in range(number_of_blocks)]
Nratio = np.concatenate([NN[i]/NN[i].sum(axis = -2, keepdims = True) for i in range(number_of_blocks)])
Nratio = np.save(folderpath + r"\Nratio.npy", Nratio)
np.save(folderpath + r"\AA.npy", AA)

In [7]:
NN = np.concatenate(NN)
NN = NN.squeeze()
NN = NN / NN.sum(axis = -1, keepdims = True)

In [8]:
df_sipm_ratio = pd.DataFrame({"sipm"+str(i)+"ratio":NN[:,i] for i in range(NN.shape[1])})
df_sipm_ratio[quantity_of_interest] = df[quantity_of_interest].values


In [9]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

# add traces

traces = []
for i in range(NN.shape[1]):
    traces.append(go.Scatter(x=df_sipm_ratio[quantity_of_interest], y=df_sipm_ratio["sipm"+str(i)+"ratio"], mode='markers', name="sipm"+str(i)+"ratio"))
fig = go.Figure(data=traces)


In [10]:
fig